# Pipeline ETL de dados da Web utilizando Python



Um pipeline ETL de dados é um conjunto de processos organizados e automatizados para extrair, transformar e carregar (ETL) dados de várias fontes para um destino específico, geralmente um armazenamento de dados ou um data warehouse. Esses pipelines são amplamente utilizados para movimentar e preparar dados, os preparando para análises, relatórios e outras aplicações.


1.   **<font color="#f4a261">Extração (Extract):</font>** Nesta etapa, os dados são coletados de diversas fontes, como bancos de dados, planilhas, APIs, arquivos CSV, entre outros. A extração pode envolver a leitura direta dos dados ou a replicação de dados de sistemas de origem para o sistema de destino.
2.  **<font color="#f4a261"> Transformação (Transform): </font>** Os dados extraídos raramente estão prontos para análise imediatamente. A etapa de transformação envolve a aplicação de diversos processos para limpar, filtrar, combinar e formatar os dados de acordo com as necessidades da análise. Transformações também podem incluir cálculos, conversões de formatos, renomeação de colunas e agregações.
3.  **<font color="#f4a261"> Carregamento (Load): </font>**  Nesta etapa, os dados transformados são carregados no destino final, como um data warehouse ou um banco de dados de análise. Dependendo do sistema de destino, isso pode envolver a criação de tabelas, esquemas e a definição de chaves primárias ou índices.

Ao reunir esses dados de forma organizada e prepará-los para análise, as empresas podem adquirir informações preciosas para direcionar suas decisões e estratégias com mais embasamento.





## Importando as bibliotecas

In [3]:
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import nltk
import warnings

warnings.filterwarnings('ignore')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eduardaaraujo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Extração de texto de um artigo na web


In [13]:
# Definindo a classe WebScraper e extraindo texto de um artigo:
class WebScraper:
    def __init__(self, url):
        self.url = url

    def extract_article_text(self):
        # Fazendo uma requisição HTTP GET para a URL fornecida:
        response = requests.get(self.url)

        # Obtendo o conteúdo HTML:
        html_content = response.content

        # Criando um objeto BeautifulSoup para analisar o HTML:
        soup = BeautifulSoup(html_content, "html.parser")

        # Extraindo o texto do artigo:
        article_text = soup.get_text()

        # Retornando o texto do artigo:
        return article_text

O objetivo é armazenar a frequência de cada palavra no artigo. Para isso, é necessário limpar e pré-processar o texto extraído do artigo.

## Processador de texto


In [14]:
class TextProcessor:
    def __init__(self, nltk_stopwords):
        self.nltk_stopwords = nltk_stopwords

    def tokenize_and_clean(self, text):
        words = text.split()
        filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in self.nltk_stopwords]
        return filtered_words

* 'nltk_stopwords': são as stopwords do NLTK (Natural Language Toolkit). Stopwords são palavras comuns, como "a", "an", "the", "is", "and", etc., que geralmente não carregam muita informação em análises de texto e, portanto, podem ser removidas.


* 'tokenize_and_clean': aceita um argumento chamado text, que é o texto que precisa ser processado.



> 1.   'words = text.split()': divide o texto em palavras individuais com base nos espaços em branco. Isso cria uma lista de palavras.
2.   'filtered_words = [...]': é uma compreensão de lista que itera sobre cada palavra na lista words. O loop verifica duas condições para cada palavra:
  *   'word.isalpha()': Verifica se a palavra contém apenas caracteres alfabéticos (letras) e não contém números ou caracteres especiais.
  *   'word.lower() not in self.nltk_stopwords':  Verifica se a versão em letras minúsculas da palavra não está presente na lista de stopwords fornecida durante a inicialização da classe.
3.   'word.lower()': converte cada palavra para letras minúsculas. Isso ajuda a tratar palavras em maiúsculas e minúsculas como equivalentes.


O método devolve uma lista de palavras limpas e processadas, excluindo stopwords e palavras não alfabéticas.



## Processo ETL

In [15]:
class ETLPipeline:
    def __init__(self, url):
        self.url = url
        self.nltk_stopwords = set(stopwords.words("english"))

    def run(self):
        scraper = WebScraper(self.url)
        article_text = scraper.extract_article_text()

        processor = TextProcessor(self.nltk_stopwords)
        filtered_words = processor.tokenize_and_clean(article_text)

        word_freq = Counter(filtered_words)
        df = pd.DataFrame(word_freq.items(), columns=["Palavras", "Frequências"])
        df = df.sort_values(by="Frequências", ascending=False)
        return df

Foi feita a extração de um texto de um artigo da web, processamento para contar a frequência das palavras e retorno dos resultados em um DataFrame ordenado.

## Execução do Pipeline


Exemplo 1: Artigo "The Proven Path to Doing Unique and Meaningful Work" por James Clear

In [18]:
if __name__ == "__main__":
    article_url = "https://jamesclear.com/stay-on-the-bus"
    pipeline = ETLPipeline(article_url)
    result_df = pipeline.run()
    print(result_df.head(10))

     Palavras  Frequências
41        bus           25
5        work           15
80      years            9
191    better            7
210     ideas            7
42    station            7
137       get            7
0        stay            6
40   helsinki            6
25        new            6


Exemplo 2: Artigo "Athleisure, barre and kale: the tyranny of the ideal woman"

In [27]:
if __name__ == "__main__":
    article_url = "https://www.theguardian.com/news/2019/aug/02/athleisure-barre-kale-tyranny-ideal-woman-labour"
    pipeline = ETLPipeline(article_url)
    result_df = pipeline.run()
    print(result_df.head(20))

       Palavras  Frequências
124        like           21
3         woman           19
2         ideal           18
4         women           16
0         barre           14
151        even           13
214        life           12
273        time           12
203    exercise           11
160         way           11
830  athleisure            9
101        many            9
97         work            9
216       feels            8
150        look            8
102      people            8
275      social            7
62       always            7
48         long            7
265          us            7


# Conclusão

Um pipeline ETL (Extração, Transformação e Carregamento) de dados da Web é um procedimento organizado e sistemático empregado na engenharia de dados para adquirir, processar e integrar informações vindas de diversas fontes online, as transformando em dados estruturados e acessíveis. Esse processo possibilita a análise eficiente e o armazenamento desses dados, contribuindo para insights valiosos e tomadas de decisão fundamentadas.

## Possíveis casos de uso




1.   Monitoramento de Mídias Sociais: Empresas podem coletar e analisar dados de redes sociais para entender sentimentos do público, acompanhar tendências e melhorar estratégias de marketing.

2.   Coleta de Dados Financeiros: Para análises de mercado, empresas podem extrair dados financeiros, como preços de ações, taxas de câmbio e indicadores econômicos, de várias fontes online.

3.   Raspagem de Dados para Pesquisa: Pesquisadores podem usar pipelines ETL para coletar dados relevantes para suas pesquisas acadêmicas de fontes na web.

4.   Acompanhamento de Preços e Produtos: Empresas de e-commerce podem extrair dados de preços e informações de produtos de sites concorrentes para ajustar suas estratégias de preços.

5.   Análise de Notícias e Opiniões: Empresas podem coletar e processar notícias, artigos e opiniões da web para entender a percepção do público e tomar decisões informadas.

6.   Dados de Sensoriamento Remoto: Para monitorar condições ambientais, dados de sensoriamento remoto de satélites e estações meteorológicas podem ser coletados e transformados.

